In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, LeavePOut
#from google.colab import drive
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis
#drive.mount('/content/gdrive/')
#df=pd.read_csv('/content/gdrive/MyDrive/백병원_프로젝트/ver_2_for_imputation_selected.csv')
df=pd.read_csv('./ver_2_for_imputation_selected.csv')
df.head()

,Temp_No,Age,Sex,Death,Status,Cancer,Dur_Event_day,Dur_Censored_day,Surtime_day,Etiology,...,smoking,alcohol,Cortical_infarct,RoPE_score,RoPE_score_5_scale,Cancer.1,composite_coagulopathy,Plt_diagnosis,Hb_diagnosis,CRP_diagnosis
0,1,55,0,NaN,0,0,NaN,3170.0,3170,7,...,0,1,1,7,4,0,0,158.0,11.9,0.43
1,2,75,0,NaN,0,0,NaN,2451.0,2451,7,...,0,0,0,3,2,0,0,203.0,12.8,0.21
2,3,81,0,NaN,0,1,NaN,551.0,551,7,...,0,0,0,4,2,1,1,223.0,10.8,0.12
3,4,44,1,NaN,1,0,5.0,NaN,5,7,...,0,1,1,8,4,0,0,199.0,15.2,0.14
4,5,71,1,1.0,1,0,603.0,NaN,603,4,...,0,1,0,3,2,0,1,232.0,10.0,5.68


In [3]:
df=df.dropna(subset=['d_dimer_num_diagnosis'],axis=0)
df.drop(['Temp_No','Cancer.1'],axis=1, inplace=True)
df.drop(['Death','Dur_Event_day','Dur_Censored_day'],axis=1, inplace=True)
df['size'].fillna('Nan', inplace=True)
df['Amt_TEE'].fillna('Nan', inplace=True)
df['Beat '].fillna('Nan', inplace=True)
df['Athero'].fillna('Nan', inplace=True)
df['oldest_d_dimer_num'].fillna(df['oldest_d_dimer_num'].mean(), inplace=True)
df['Homocysteine'].fillna(df['Homocysteine'].mean(), inplace=True)
df['HbA1c'].fillna(df['HbA1c'].mean(), inplace=True)
df['Chol'].fillna(df['Chol'].mean(), inplace=True)
df['HDL'].fillna(df['HDL'].mean(), inplace=True)
df['LDL'].fillna(df['LDL'].mean(), inplace=True)
df['TG'].fillna(df['TG'].mean(), inplace=True)
df['CRP_diagnosis'].fillna(df['CRP_diagnosis'].mean(), inplace=True)

from sklearn.preprocessing import LabelEncoder
def labelencoder(df):
  for c in df.columns:
    if df[c].dtype=='object':
      encoder=LabelEncoder()
      df[c]=encoder.fit_transform(list(df[c].values))
  return df

labelencoder(df)
df.head()

,Age,Sex,Status,Cancer,Surtime_day,Etiology,AF_composite,PFO_positive,size,Amt_TEE,...,TG,smoking,alcohol,Cortical_infarct,RoPE_score,RoPE_score_5_scale,composite_coagulopathy,Plt_diagnosis,Hb_diagnosis,CRP_diagnosis
0,55,0,0,0,3170,7,0,1,48,3,...,71.000000,0,1,1,7,4,0,158.0,11.9,0.43
2,81,0,0,1,551,7,0,0,0,0,...,127.777358,0,0,0,4,2,1,223.0,10.8,0.12
4,71,1,1,0,603,4,0,0,0,0,...,127.777358,0,1,0,3,2,1,232.0,10.0,5.68
5,61,0,1,0,181,7,0,0,0,0,...,82.000000,1,1,1,5,3,0,316.0,14.8,0.10
6,49,1,0,1,955,4,0,1,48,2,...,127.777358,0,1,0,7,4,1,169.0,15.0,0.10


In [4]:
def get_boolean(x):
  value=''
  if x==0: value=False
  elif x==1: value=True
  return value

feature=df.drop(['Status','Surtime_day'], axis=1)
#feature_pos=df1.drop(['Status','Surtime_day'], axis=1)

label=df[['Status','Surtime_day']]
#label_pos=df1[['Status','Surtime_day']]

status=list(label['Status'].values)
#status_pos=list(label_pos['Status'].values)

duration=list(label['Surtime_day'].values.astype(int))
#duration_pos=list(label_pos['Surtime_day'].values.astype(int))

st_label=np.array([(status[0],duration[0])], dtype=[('Status','?'),('Duration','<f8')])
#st_label_pos=np.array([(status_pos[0],duration_pos[0])], dtype=[('Status','?'),('Duration','<f8')])

for c in range(1,len(label.index)):
  lbl=np.array([(status[c],duration[c])], dtype=[('Status','?'),('Duration','<f8')])
  st_label=np.append(st_label,lbl,axis=0)

#for c in range(1,len(label_pos.index)):
#  lbl=np.array([(status_pos[c],duration_pos[c])], dtype=[('Status','?'),('Duration','<f8')])
#  st_label_pos=np.append(st_label_pos,lbl,axis=0)

In [5]:
from sksurv.ensemble import RandomSurvivalForest,GradientBoostingSurvivalAnalysis,ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis,CoxnetSurvivalAnalysis

rsf=RandomSurvivalForest(random_state=10, n_estimators=100,bootstrap=True, max_depth=6, max_features='log2', min_samples_leaf=8, min_samples_split=2)
coxph=CoxPHSurvivalAnalysis(alpha=0.8978654438751634, n_iter= 4)
gbsurv=GradientBoostingSurvivalAnalysis(loss='coxph', n_estimators=184, random_state=50, max_depth=1, min_samples_leaf=2, learning_rate=1.0, min_samples_split=5)
lasso=CoxnetSurvivalAnalysis(l1_ratio=1.0, alpha_min_ratio= 0.0039738792396125265, max_iter= 568536,n_alphas= 127, fit_baseline_model=True)
elastic=CoxnetSurvivalAnalysis(l1_ratio=0.12945630593570634, alpha_min_ratio=0.0022290682393707553,n_alphas=127, max_iter=339808, fit_baseline_model=True)

X_train,X_test,y_train,y_test=train_test_split(feature,st_label,random_state=50,test_size=0.2, shuffle=True)
lpo=LeavePOut(p=6)
lpo.get_n_splits(X_train)

581246946280

In [9]:
coxph_scores=[]
for train_index, val_index in lpo.split(X_train):
    x_tr,x_val=X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr,y_val=y_train[train_index], y_train[val_index]
    rsf.fit(x_tr,y_tr)
    score=rsf.score(x_val,y_val)
    print(val_index, score)
    coxph_scores.append(score)

print('RandomSurvivalForest cindexes: ',coxph_scores)
print('Mean c-index: ', np.round(np.mean(coxph_scores),3))

[0 1 2 3 4 5] 0.5555555555555556
[0 1 2 3 4 6] 0.5555555555555556
[0 1 2 3 4 7] 0.4444444444444444
[0 1 2 3 4 8] 0.4444444444444444
[0 1 2 3 4 9] 0.5833333333333334
[ 0  1  2  3  4 10] 0.4444444444444444
[ 0  1  2  3  4 11] 0.5555555555555556
[ 0  1  2  3  4 12] 0.3333333333333333
[ 0  1  2  3  4 13] 0.5555555555555556
[ 0  1  2  3  4 14] 0.5555555555555556
[ 0  1  2  3  4 15] 0.5555555555555556
[ 0  1  2  3  4 16] 0.6666666666666666
[ 0  1  2  3  4 17] 0.4444444444444444
[ 0  1  2  3  4 18] 0.5555555555555556
[ 0  1  2  3  4 19] 0.4444444444444444
[ 0  1  2  3  4 20] 0.6666666666666666
[ 0  1  2  3  4 21] 0.6666666666666666
[ 0  1  2  3  4 22] 0.6666666666666666
[ 0  1  2  3  4 23] 0.6666666666666666
[ 0  1  2  3  4 24] 0.4444444444444444
[ 0  1  2  3  4 25] 0.6666666666666666
[ 0  1  2  3  4 26] 0.6666666666666666
[ 0  1  2  3  4 27] 0.5555555555555556
[ 0  1  2  3  4 28] 0.5555555555555556
[ 0  1  2  3  4 29] 0.5555555555555556
[ 0  1  2  3  4 30] 0.4444444444444444
[ 0  1  2  3  4 

[  0   1   2   3   4 210] 0.6666666666666666
[  0   1   2   3   4 211] 0.4444444444444444
[  0   1   2   3   4 212] 0.5
[  0   1   2   3   4 213] 0.4444444444444444
[  0   1   2   3   4 214] 0.4444444444444444
[  0   1   2   3   4 215] 0.4444444444444444
[  0   1   2   3   4 216] 0.4444444444444444
[  0   1   2   3   4 217] 0.7777777777777778
[  0   1   2   3   4 218] 0.6666666666666666
[  0   1   2   3   4 219] 0.6666666666666666
[  0   1   2   3   4 220] 0.3333333333333333
[  0   1   2   3   4 221] 0.6666666666666666
[  0   1   2   3   4 222] 0.6666666666666666
[  0   1   2   3   4 223] 0.4444444444444444
[  0   1   2   3   4 224] 0.6666666666666666
[  0   1   2   3   4 225] 0.6666666666666666
[  0   1   2   3   4 226] 0.3333333333333333
[  0   1   2   3   4 227] 0.375
[  0   1   2   3   4 228] 0.5555555555555556
[  0   1   2   3   4 229] 0.6666666666666666
[  0   1   2   3   4 230] 0.6666666666666666
[  0   1   2   3   4 231] 0.4444444444444444
[  0   1   2   3   4 232] 0.6666666666

[  0   1   2   3   5 145] 0.4444444444444444
[  0   1   2   3   5 146] 0.25
[  0   1   2   3   5 147] 0.4444444444444444
[  0   1   2   3   5 148] 0.4444444444444444
[  0   1   2   3   5 149] 0.2222222222222222
[  0   1   2   3   5 150] 0.36363636363636365
[  0   1   2   3   5 151] 0.5555555555555556
[  0   1   2   3   5 152] 0.5555555555555556
[  0   1   2   3   5 153] 0.4166666666666667
[  0   1   2   3   5 154] 0.36363636363636365
[  0   1   2   3   5 155] 0.1111111111111111
[  0   1   2   3   5 156] 0.3333333333333333
[  0   1   2   3   5 157] 0.3333333333333333
[  0   1   2   3   5 158] 0.3333333333333333
[  0   1   2   3   5 159] 0.3333333333333333
[  0   1   2   3   5 160] 0.2222222222222222
[  0   1   2   3   5 161] 0.2222222222222222
[  0   1   2   3   5 162] 0.1111111111111111
[  0   1   2   3   5 163] 0.1111111111111111
[  0   1   2   3   5 164] 0.3333333333333333
[  0   1   2   3   5 165] 0.0
[  0   1   2   3   5 166] 0.3333333333333333
[  0   1   2   3   5 167] 0.333333333

[ 0  1  2  3  6 76] 0.5555555555555556
[ 0  1  2  3  6 77] 0.5555555555555556
[ 0  1  2  3  6 78] 0.7777777777777778
[ 0  1  2  3  6 79] 0.5555555555555556
[ 0  1  2  3  6 80] 0.7777777777777778
[ 0  1  2  3  6 81] 0.7777777777777778
[ 0  1  2  3  6 82] 0.7777777777777778
[ 0  1  2  3  6 83] 0.5555555555555556
[ 0  1  2  3  6 84] 0.7777777777777778
[ 0  1  2  3  6 85] 0.7777777777777778
[ 0  1  2  3  6 86] 0.5555555555555556
[ 0  1  2  3  6 87] 0.625
[ 0  1  2  3  6 88] 0.7777777777777778
[ 0  1  2  3  6 89] 0.7777777777777778
[ 0  1  2  3  6 90] 0.5555555555555556
[ 0  1  2  3  6 91] 0.5
[ 0  1  2  3  6 92] 0.4444444444444444
[ 0  1  2  3  6 93] 0.5555555555555556
[ 0  1  2  3  6 94] 0.5555555555555556
[ 0  1  2  3  6 95] 0.4444444444444444
[ 0  1  2  3  6 96] 0.4444444444444444
[ 0  1  2  3  6 97] 0.4444444444444444
[ 0  1  2  3  6 98] 0.3333333333333333
[ 0  1  2  3  6 99] 0.3333333333333333
[  0   1   2   3   6 100] 0.6666666666666666
[  0   1   2   3   6 101] 0.6666666666666666
[ 

[  0   1   2   3   6 271] 0.4444444444444444
[  0   1   2   3   6 272] 0.5
[  0   1   2   3   6 273] 0.4166666666666667
[  0   1   2   3   6 274] 0.4444444444444444
[  0   1   2   3   6 275] 0.5
[0 1 2 3 7 8] 0.3333333333333333
[0 1 2 3 7 9] 0.25
[ 0  1  2  3  7 10] 0.2222222222222222
[ 0  1  2  3  7 11] 0.2222222222222222
[ 0  1  2  3  7 12] 0.2222222222222222
[ 0  1  2  3  7 13] 0.4444444444444444
[ 0  1  2  3  7 14] 0.2222222222222222
[ 0  1  2  3  7 15] 0.2222222222222222
[ 0  1  2  3  7 16] 0.4444444444444444
[ 0  1  2  3  7 17] 0.2222222222222222
[ 0  1  2  3  7 18] 0.4444444444444444
[ 0  1  2  3  7 19] 0.2222222222222222
[ 0  1  2  3  7 20] 0.4444444444444444
[ 0  1  2  3  7 21] 0.4444444444444444
[ 0  1  2  3  7 22] 0.4444444444444444
[ 0  1  2  3  7 23] 0.4444444444444444
[ 0  1  2  3  7 24] 0.2222222222222222
[ 0  1  2  3  7 25] 0.4444444444444444
[ 0  1  2  3  7 26] 0.4444444444444444
[ 0  1  2  3  7 27] 0.4444444444444444
[ 0  1  2  3  7 28] 0.4444444444444444
[ 0  1  2  3

...

[ 0  1  2  3 12 93] 0.5555555555555556
[ 0  1  2  3 12 94] 0.5555555555555556
[ 0  1  2  3 12 95] 0.5555555555555556
[ 0  1  2  3 12 96] 0.3333333333333333
[ 0  1  2  3 12 97] 0.2222222222222222
[ 0  1  2  3 12 98] 0.25
[ 0  1  2  3 12 99] 0.2222222222222222
[  0   1   2   3  12 100] 0.4444444444444444
[  0   1   2   3  12 101] 0.4444444444444444
[  0   1   2   3  12 102] 0.2222222222222222
[  0   1   2   3  12 103] 0.4444444444444444
[  0   1   2   3  12 104] 0.25
[  0   1   2   3  12 105] 0.3333333333333333
[  0   1   2   3  12 106] 0.3333333333333333
[  0   1   2   3  12 107] 0.3333333333333333
[  0   1   2   3  12 108] 0.3333333333333333
[  0   1   2   3  12 109] 0.4444444444444444
[  0   1   2   3  12 110] 0.25
[  0   1   2   3  12 111] 0.4444444444444444
[  0   1   2   3  12 112] 0.1111111111111111
[  0   1   2   3  12 113] 0.3333333333333333
[  0   1   2   3  12 114] 0.2222222222222222
[  0   1   2   3  12 115] 0.1111111111111111
[  0   1   2   3  12 116] 0.3333333333333333
[  0

[ 0  1  2  3 13 24] 0.4444444444444444
[ 0  1  2  3 13 25] 0.5555555555555556
[ 0  1  2  3 13 26] 0.5555555555555556
[ 0  1  2  3 13 27] 0.5555555555555556
[ 0  1  2  3 13 28] 0.5555555555555556
[ 0  1  2  3 13 29] 0.6666666666666666
[ 0  1  2  3 13 30] 0.3333333333333333
[ 0  1  2  3 13 31] 0.3333333333333333
[ 0  1  2  3 13 32] 0.5
[ 0  1  2  3 13 33] 0.5
[ 0  1  2  3 13 34] 0.5555555555555556
[ 0  1  2  3 13 35] 0.5555555555555556
[ 0  1  2  3 13 36] 0.5
[ 0  1  2  3 13 37] 0.5555555555555556
[ 0  1  2  3 13 38] 0.4444444444444444
[ 0  1  2  3 13 39] 0.375
[ 0  1  2  3 13 40] 0.5
[ 0  1  2  3 13 41] 0.5555555555555556
[ 0  1  2  3 13 42] 0.3333333333333333
[ 0  1  2  3 13 43] 0.4444444444444444
[ 0  1  2  3 13 44] 0.4166666666666667
[ 0  1  2  3 13 45] 0.4444444444444444
[ 0  1  2  3 13 46] 0.4444444444444444
[ 0  1  2  3 13 47] 0.4444444444444444
[ 0  1  2  3 13 48] 0.5555555555555556
[ 0  1  2  3 13 49] 0.3333333333333333
[ 0  1  2  3 13 50] 0.6666666666666666
[ 0  1  2  3 13 51] 

[  0   1   2   3  13 224] 0.6666666666666666
[  0   1   2   3  13 225] 0.7777777777777778
[  0   1   2   3  13 226] 0.5555555555555556
[  0   1   2   3  13 227] 0.5
[  0   1   2   3  13 228] 0.5555555555555556
[  0   1   2   3  13 229] 0.6666666666666666
[  0   1   2   3  13 230] 0.5555555555555556
[  0   1   2   3  13 231] 0.3333333333333333
[  0   1   2   3  13 232] 0.5555555555555556
[  0   1   2   3  13 233] 0.36363636363636365
[  0   1   2   3  13 234] 0.4444444444444444
[  0   1   2   3  13 235] 0.2222222222222222
[  0   1   2   3  13 236] 0.4444444444444444
[  0   1   2   3  13 237] 0.4444444444444444
[  0   1   2   3  13 238] 0.375
[  0   1   2   3  13 239] 0.5
[  0   1   2   3  13 240] 0.4444444444444444
[  0   1   2   3  13 241] 0.5555555555555556
[  0   1   2   3  13 242] 0.5555555555555556
[  0   1   2   3  13 243] 0.5555555555555556
[  0   1   2   3  13 244] 0.4444444444444444
[  0   1   2   3  13 245] 0.4166666666666667
[  0   1   2   3  13 246] 0.6363636363636364
[  0   

[  0   1   2   3  14 166] 0.4166666666666667
[  0   1   2   3  14 167] 0.3333333333333333
[  0   1   2   3  14 168] 0.2222222222222222
[  0   1   2   3  14 169] 0.4444444444444444
[  0   1   2   3  14 170] 0.2222222222222222
[  0   1   2   3  14 171] 0.2222222222222222
[  0   1   2   3  14 172] 0.25
[  0   1   2   3  14 173] 0.3333333333333333
[  0   1   2   3  14 174] 0.25
[  0   1   2   3  14 175] 0.4444444444444444
[  0   1   2   3  14 176] 0.4444444444444444
[  0   1   2   3  14 177] 0.2222222222222222
[  0   1   2   3  14 178] 0.2222222222222222
[  0   1   2   3  14 179] 0.375
[  0   1   2   3  14 180] 0.5
[  0   1   2   3  14 181] 0.16666666666666666
[  0   1   2   3  14 182] 0.3333333333333333
[  0   1   2   3  14 183] 0.4444444444444444
[  0   1   2   3  14 184] 0.375
[  0   1   2   3  14 185] 0.4166666666666667
[  0   1   2   3  14 186] 0.4444444444444444
[  0   1   2   3  14 187] 0.1111111111111111
[  0   1   2   3  14 188] 0.3333333333333333
[  0   1   2   3  14 189] 0.22222

KeyboardInterrupt: 

In [ ]:
scores=[]
for train_index, val_index in lpo.split(X_train):
  x_tr,x_val=X_train.iloc[train_index], X_train.iloc[val_index]
  y_tr,y_val=y_train[train_index], y_train[val_index]
  coxph.fit(x_tr,y_tr)
  score=coxph.score(x_val,y_val)
  scores.append(score)

print('coxPHSurvivalAnalysis cindexes: ',coxph_scores)
print('Mean c-index: ', np.round(np.mean(coxph_scores),3))

In [ ]:
scores=[]
for train_index, val_index in lpo.split(X_train):
  x_tr,x_val=X_train.iloc[train_index], X_train.iloc[val_index]
  y_tr,y_val=y_train[train_index], y_train[val_index]
  gbsurv.fit(x_tr,y_tr)
  score=gbsurv.score(x_val,y_val)
  scores.append(score)

print('GradientBoostingSurvivalAnalysis cindexes: ',coxph_scores)
print('Mean c-index: ', np.round(np.mean(coxph_scores),3))

In [ ]:
scores=[]
for train_index, val_index in lpo.split(X_train):
  x_tr,x_val=X_train.iloc[train_index], X_train.iloc[val_index]
  y_tr,y_val=y_train[train_index], y_train[val_index]
  lasso.fit(x_tr,y_tr)
  score=lasso.score(x_val,y_val)
  scores.append(score)

print('LassoSurvivalAnalysis cindexes: ',coxph_scores)
print('Mean c-index: ', np.round(np.mean(coxph_scores),3))

In [ ]:
scores=[]
for train_index, val_index in lpo.split(X_train):
  x_tr,x_val=X_train.iloc[train_index], X_train.iloc[val_index]
  y_tr,y_val=y_train[train_index], y_train[val_index]
  elastic.fit(x_tr,y_tr)
  score=elastic.score(x_val,y_val)
  scores.append(score)

print('ElasticSurvivalAnalysis cindexes: ',coxph_scores)
print('Mean c-index: ', np.round(np.mean(coxph_scores),3))